In [4]:
#Import necessary packages
import os
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import re
from datetime import datetime
import xarray as xr
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import glob
from netCDF4 import Dataset
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim, xy, ll_to_xy,
                 cartopy_ylim, latlon_coords, CoordPair, GeoBounds, interp2dxy)
import xesmf as xe
import pandas as pd
import proplot as pplt
import cartopy.crs as crs

In [5]:
# Define date range
dates = pd.date_range('2017-01-01','2019-12-31')

In [6]:
# Read in just the first file
pcpfirst = xr.open_dataset('/glade/derecho/scratch/cab478/IMERG/3B-HHR.MS.MRG.3IMERG.20170101-S000000-E002959.0000.V07B.HDF5.nc4')['precipitation']

In [67]:
# Read wrfinput file used in Noah-MP simulations to extract latitudes/longitudes
ncfile = Dataset('/glade/u/home/cab478/hrldas_resources/resources/wrfinput_d01')
dummy  = getvar(ncfile, "SHDMAX")
lats, lons = latlon_coords(dummy)

In [70]:
# Define output (WRF) grid as an xarray Dataset
grid_out = xr.Dataset({'lat': (['south_north','west_east'], lats.values),
                       'lon': (['south_north','west_east'], lons.values)})

In [ ]:
# Apply regrdding algorithm
regridder = xe.Regridder(pcpfirst, grid_out, "bilinear")

In [ ]:
# Read multiple IMERG half hourly precipitation rate files, one date at a time
# Regrid IMERG data to our model grid 
# Resample to 3-hourly frequency
# Write to NetCDF file

for d in dates:
    files = '/glade/derecho/scratch/cab478/IMERG/3B-HHR.MS.MRG.3IMERG.' + d.strftime('%Y%m%d') + '*.nc4'
    pcpday = xr.open_mfdataset(sorted(glob.glob(files)))['precipitation']
    pcpday_regrid = regridder(pcpday, keep_attrs=True)
    pcpday_3hr    = pcpday_regrid.resample(time='3H').mean()
    print(d)
    pcpday_3hr.to_netcdf('/glade/derecho/scratch/cab478/IMERG/3hr/imerg_3hr_' + d.strftime('%Y%m%d') + '.nc')